In [1]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle (1).json to kaggle (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
!mkdir -p ~/.kaggle
!cp "kaggle (1).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json



In [3]:
!kaggle datasets download -d  phucthaiv02/butterfly-image-classification


Dataset URL: https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
License(s): CC0-1.0
 52% 117M/226M [00:00<00:00, 1.23GB/s]
100% 226M/226M [00:00<00:00, 808MB/s] 


In [4]:
!unzip -q butterfly-image-classification.zip -d '/content/drive/MyDrive/datasets'

In [1]:
import tensorflow as tf
strategy = tf.distribute.get_strategy()
print("Using:", tf.config.list_physical_devices())


Using: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
!cp -r /content/drive/MyDrive/datasets/train /content/train_local

In [5]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
train_df = pd.read_csv("/content/drive/MyDrive/datasets/Training_set.csv")
test_df = pd.read_csv("/content/drive/MyDrive/datasets/Testing_set.csv")

print("Unique classes:", train_df['label'].nunique())
print(train_df['label'].value_counts())


Unique classes: 75
label
MOURNING CLOAK    131
SLEEPY ORANGE     107
ATALA             100
BROWN SIPROETA     99
SCARCE SWALLOW     97
                 ... 
AMERICAN SNOOT     74
GOLD BANDED        73
MALACHITE          73
CRIMSON PATCH      72
WOOD SATYR         71
Name: count, Length: 75, dtype: int64


In [12]:
train_df.head(
)

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART


In [13]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory="/content/drive/MyDrive/datasets/train",
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory="/content/drive/MyDrive/datasets/train",
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 3113 validated image filenames belonging to 75 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 2608 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 778 validated image filenames belonging to 75 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 2608 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze base layers

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_gen.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/model.keras",
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1)


In [16]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint, early_stop]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.0302 - loss: 19.6992
Epoch 1: val_accuracy improved from -inf to 0.05913, saving model to /content/drive/MyDrive/model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 1120s 11s/step - accuracy: 0.0302 - loss: 19.6022 - val_accuracy: 0.0591 - val_loss: 4.2143
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.0574 - loss: 4.0823
Epoch 2: val_accuracy improved from 0.05913 to 0.10283, saving model to /content/drive/MyDrive/model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 21s 208ms/step - accuracy: 0.0575 - loss: 4.0819 - val_accuracy: 0.1028 - val_loss: 4.0245
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.1159 - loss: 3.7284
Epoch 3: val_accuracy improved from 0.10283 to 0.14524, saving model to /content/drive/MyDrive/model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 23s 233ms/step - accuracy: 0.1158 - loss: 3.7287 - val_accuracy: 0.1452 - val_loss: 3.8692
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.1504

In [17]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/model.keras")

In [18]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("/content/drive/MyDrive/model.keras", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-7)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint, early_stop, lr_reduce]
)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.8261 - loss: 0.5535
Epoch 1: val_accuracy improved from -inf to 0.71851, saving model to /content/drive/MyDrive/model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 24s 242ms/step - accuracy: 0.8261 - loss: 0.5537 - val_accuracy: 0.7185 - val_loss: 1.6098 - learning_rate: 1.2500e-05
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.8500 - loss: 0.4549
Epoch 2: val_accuracy did not improve from 0.71851
98/98 ━━━━━━━━━━━━━━━━━━━━ 38s 217ms/step - accuracy: 0.8500 - loss: 0.4550 - val_accuracy: 0.7172 - val_loss: 1.6354 - learning_rate: 1.2500e-05
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.8316 - loss: 0.5423
Epoch 3: val_accuracy did not improve from 0.71851

Epoch 3: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
98/98 ━━━━━━━━━━━━━━━━━━━━ 22s 223ms/step - accuracy: 0.8317 - loss: 0.5419 - val_accuracy: 0.7134 - val_loss: 1.6581 - learning_rate: 1.2500e-05
Epoch 4/10
98/98 ━━━━━

In [21]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_gen = train_datagen.flow_from_dataframe(
    test_df,
    directory="/content/drive/MyDrive/datasets/test",
    x_col='filename',
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
)

Found 2786 validated image filenames.


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img('/content/drive/MyDrive/test.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
predicted_class = np.argmax(preds)
class_labels = list(train_gen.class_indices.keys())
print("Predicted class:", class_labels[predicted_class], f"(confidence: {np.max(preds):.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: AN 88 (confidence: 1.00)


In [23]:
import json
with open("class_indices.json", "w") as f:
    json.dump(train_gen.class_indices, f)
